# Python et la logique SQL

[SQL](https://fr.wikipedia.org/wiki/Structured_Query_Language) permet de créer des tables, de rechercher, d'ajouter, de modifier ou de supprimer des données dans les bases de données. Un peu ce que vous ferez bientôt tous les jours. C'est un langage de management de données, pas de nettoyage, d’analyse ou de statistiques avancées.

In [ ]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

Les instructions SQL s'écrivent d'une manière qui ressemble à celle de phrases ordinaires en anglais. Cette ressemblance voulue vise à faciliter l'apprentissage et la lecture. Il est néanmoins important de respecter un ordre pour les différentes instructions.

Dans ce TD, nous allons écrire des commandes en SQL via Python.

Pour plus de précisions sur SQL et les commandes qui existent, rendez-vous là [SQL, PRINCIPES DE BASE](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx3/ext2a/sql_doc.html).

## Se connecter à une base de données

A la différence des tables qu'on utilise habituellement, la base de données n'est pas visible directement en ouvrant Excel ou un éditeur de texte. Pour avoir une vue de ce que contient la base de données, il est nécessaire d'avoir un autre type de logiciel.

Pour le TD, nous vous recommandans d'installer SQLLiteSpy (disponible à cette adresse [SqliteSpy](http://www.yunqa.de/delphi/products/sqlitespy/index) ou [sqlite_bro](https://pypi.python.org/pypi/sqlite_bro) si vous voulez voir à quoi ressemble les données avant de les utiliser avec Python.

In [ ]:
import sqlite3
# on va se connecter à une base de données SQL vide
# SQLite stocke la BDD dans un simple fichier
filepath = "./DataBase.db"
open(filepath, 'w').close() #crée un fichier vide
CreateDataBase = sqlite3.connect(filepath)

QueryCurs = CreateDataBase.cursor()

La méthode [cursor](https://docs.python.org/3.5/library/sqlite3.html#sqlite3.Connection.cursor) est un peu particulière : 

Il s'agit d'une sorte de tampon mémoire intermédiaire, destiné à mémoriser temporairement les données en cours de traitement, ainsi que les opérations que vous effectuez sur elles, avant leur transfert définitif dans la base de données. Tant que la  méthode [commit](https://docs.python.org/3.5/library/sqlite3.html#sqlite3.Connection.commit) n'aura pas été appelée, aucun ordre ne sera appliqué à la base de données.

--------------------
A présent que nous sommes connectés à la base de données, on va créer une table qui contient plusieurs variables de format différents
- ID sera la clé primaire de la base
- Nom, Rue, Ville, Pays seront du text
- Prix sera un réel

In [ ]:
# On définit une fonction de création de table
def CreateTable(nom_bdd):
    QueryCurs.execute('''CREATE TABLE IF NOT EXISTS ''' + nom_bdd + '''
    (id INTEGER PRIMARY KEY, Name TEXT,City TEXT, Country TEXT, Price REAL)''')

# On définit une fonction qui permet d'ajouter des observations dans la table    
def AddEntry(nom_bdd, Nom,Ville,Pays,Prix):
    QueryCurs.execute('''INSERT INTO ''' + nom_bdd + ''' 
    (Name,City,Country,Price) VALUES (?,?,?,?)''',(Nom,Ville,Pays,Prix))
    
def AddEntries(nom_bdd, data):
    """ data : list with (Name,City,Country,Price) tuples to insert
    """
    QueryCurs.executemany('''INSERT INTO ''' + nom_bdd + ''' 
    (Name,City,Country,Price) VALUES (?,?,?,?)''',data)
    
    
### On va créer la table clients

CreateTable('Clients')

AddEntry('Clients','Toto','Munich','Germany',5.2)
AddEntries('Clients',
    [('Bill','Berlin','Germany',2.3),
    ('Tom','Paris','France',7.8),
    ('Marvin','Miami','USA',15.2),
    ('Anna','Paris','USA',7.8)])

# on va "commit" c'est à dire qu'on va valider la transaction. 
# > on va envoyer ses modifications locales vers le référentiel central - la base de données SQL

CreateDataBase.commit()

### Voir la table
Pour voir ce qu'il y a dans la table, on utilise un premier Select où on demande à voir toute la table

In [ ]:
QueryCurs.execute('SELECT * FROM Clients')
Values = QueryCurs.fetchall()
print(Values)

[(1, 'Toto', 'Munich', 'Germany', 5.2), (2, 'Bill', 'Berlin', 'Germany', 2.3), (3, 'Tom', 'Paris', 'France', 7.8), (4, 'Marvin', 'Miami', 'USA', 15.2), (5, 'Anna', 'Paris', 'USA', 7.8)]


### Passer en pandas

Rien de plus simple : plusieurs manières de faire

In [ ]:
import pandas as pd
# méthode SQL Query
df1 = pd.read_sql_query('SELECT * FROM Clients', CreateDataBase)
print("En utilisant la méthode read_sql_query \n", df1.head(), "\n")


#méthode DataFrame en utilisant la liste issue de .fetchall()
df2 = pd.DataFrame(Values, columns=['ID','Name','City','Country','Price'])
print("En passant par une DataFrame \n", df2.head())

En utilisant la méthode read_sql_query 
    id    Name    City  Country  Price
0   1    Toto  Munich  Germany    5.2
1   2    Bill  Berlin  Germany    2.3
2   3     Tom   Paris   France    7.8
3   4  Marvin   Miami      USA   15.2
4   5    Anna   Paris      USA    7.8 

En passant par une DataFrame 
    ID    Name    City  Country  Price
0   1    Toto  Munich  Germany    5.2
1   2    Bill  Berlin  Germany    2.3
2   3     Tom   Paris   France    7.8
3   4  Marvin   Miami      USA   15.2
4   5    Anna   Paris      USA    7.8


## Comparaison SQL et pandas
### SELECT

En SQL, la sélection se fait en utilisant des virgules ou * si on veut sélectionner toutes les colonnes

In [ ]:
# en SQL
QueryCurs.execute('SELECT ID,City FROM Clients LIMIT 2')
Values = QueryCurs.fetchall()
print(Values)

[(1, 'Munich'), (2, 'Berlin')]


En pandas, la sélection de colonnes se fait en donnant une liste

In [ ]:
#sur la table
df2[['ID','City']].head(2)

,ID,City
0,1,Munich
1,2,Berlin


### WHERE
En SQL, on utilise WHERE pour filtrer les tables selon certaines conditions

In [ ]:
QueryCurs.execute('SELECT * FROM Clients WHERE City=="Paris"')
print(QueryCurs.fetchall())

[(3, 'Tom', 'Paris', 'France', 7.8), (5, 'Anna', 'Paris', 'USA', 7.8)]


Avec Pandas, on peut utiliser plusieurs manières de faire : 
    - avec un booléen
    - en utilisant la méthode 'query'

In [ ]:
df2[df2['City'] == "Paris"]

,ID,Name,City,Country,Price
2,3,Tom,Paris,France,7.8
4,5,Anna,Paris,USA,7.8


In [ ]:
df2.query('City == "Paris"')

,ID,Name,City,Country,Price
2,3,Tom,Paris,France,7.8
4,5,Anna,Paris,USA,7.8


Pour mettre plusieurs conditions, on utilise : 
- & en Python, AND en SQL
- | en python, OR en SQL

In [ ]:
QueryCurs.execute('SELECT * FROM Clients WHERE City=="Paris" AND Country == "USA"')
print(QueryCurs.fetchall())

[(5, 'Anna', 'Paris', 'USA', 7.8)]


In [ ]:
df2.query('City == "Paris" & Country == "USA"')

df2[(df2['City'] == "Paris") & (df2['Country'] == "USA")]

,ID,Name,City,Country,Price
4,5,Anna,Paris,USA,7.8


## GROUP BY

En pandas, l'opération GROUP BY de SQL s'effectue avec une méthode similaire : [groupby](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html) 

[groupby](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html) sert à regrouper des observations en groupes selon les modalités de certaines variables en appliquant une fonction d'aggrégation sur d'autres variables.

In [ ]:
QueryCurs.execute('SELECT Country, count(*)  FROM Clients GROUP BY Country')
print(QueryCurs.fetchall())

[('France', 1), ('Germany', 2), ('USA', 2)]


Attention, en pandas, la fonction [count](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.count.html?highlight=count#pandas.core.groupby.DataFrameGroupBy.count) ne fait pas la même chose qu'en SQL. [count](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.count.html?highlight=count#pandas.core.groupby.DataFrameGroupBy.count) s'applique à toutes les colonnes et compte toutes les observations non nulles.

In [ ]:
df2.groupby('Country').count()

,ID,Name,City,Price
Country,,,,
France,1,1,1,1
Germany,2,2,2,2
USA,2,2,2,2


Pour réaliser la même chose qu'en SQL, il faut utiliser la méthode [size](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.size.html?highlight=size#pandas.core.groupby.DataFrameGroupBy.size).

In [ ]:
df2.groupby('Country').size()

Country
France     1
Germany    2
USA        2
dtype: int64

On peut aussi appliquer des fonctions plus sophistiquées lors d'un [groupby](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html)

In [ ]:
QueryCurs.execute('SELECT Country, AVG(Price), count(*)  FROM Clients GROUP BY Country')
print(QueryCurs.fetchall())

[('France', 7.8, 1), ('Germany', 3.75, 2), ('USA', 11.5, 2)]


Avec pandas, on peut appeler les fonctions classiques de [numpy](http://www.numpy.org/)

In [ ]:
import numpy as np
df2.groupby('Country').agg({'Price': np.mean, 'Country': np.size})

,Price,Country
Country,,
France,7.80,1
Germany,3.75,2
USA,11.50,2


Ou utiliser des fonctions [lambda](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions).

In [ ]:
# par exemple calculer le prix moyen et le multiplier par 2
df2.groupby('Country')['Price'].apply(lambda x: 2*x.mean())

Country
France     15.6
Germany     7.5
USA        23.0
Name: Price, dtype: float64

In [ ]:
QueryCurs.execute('SELECT Country, 2*AVG(Price) FROM Clients GROUP BY Country').fetchall()

[('France', 15.6), ('Germany', 7.5), ('USA', 23.0)]

In [ ]:
QueryCurs.execute('SELECT * FROM Clients WHERE Country == "Germany"')
print(QueryCurs.fetchall())
QueryCurs.execute('SELECT * FROM Clients WHERE City=="Berlin" AND Country == "Germany"')
print(QueryCurs.fetchall())
QueryCurs.execute('SELECT * FROM Clients WHERE Price BETWEEN 7 AND 20')
print(QueryCurs.fetchall())

[(1, 'Toto', 'Munich', 'Germany', 5.2), (2, 'Bill', 'Berlin', 'Germany', 2.3)]
[(2, 'Bill', 'Berlin', 'Germany', 2.3)]
[(3, 'Tom', 'Paris', 'France', 7.8), (4, 'Marvin', 'Miami', 'USA', 15.2), (5, 'Anna', 'Paris', 'USA', 7.8)]


## Enregistrer une table SQL sous un autre format 

On utilise le package csv, l'option 'w' pour 'write'. 

On crée l'objet "writer", qui vient du package csv.

Cet objet a deux méthodes : 
- writerow pour les noms de colonnes  : une liste
- writerows pour les lignes : un ensemble de liste


In [ ]:
data = QueryCurs.execute('SELECT * FROM Clients')

import csv

with open('./output.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerow(['id','Name','City','Country','Price'])
    writer.writerows(data)

On peut également passer par un DataFrame pandas et utiliser .to_csv()

In [ ]:
QueryCurs.execute('''DROP TABLE Clients''')
QueryCurs.close()

# Exercice

Dans cet exercice, nous allons manipuler les tables de la base de données World. 

Avant tout, télechargez le fichier et connectez vous à la base de données en utilisant [sqlite3](https://docs.python.org/3/library/sqlite3.html?highlight=sqlite3#module-sqlite3) et [connect](https://docs.python.org/3/library/sqlite3.html?highlight=sqlite3#sqlite3.connect).

Familiarisez vous avec la base de données :
- quelles sont les tables ? 
- quelles sont les variables de ces tables ? 
- utilisez la fonction PRAGMA pour obtenir des informations sur les tables
    

## Question 1
- Quels sont les 10 pays qui ont le plus de langues ?
- Quelle langue est présente dans le plus de pays ?

## Question 2
- Quelles sont les différentes formes de gouvernements dans les pays du monde ?
- Quels sont les 3 gouvernements où la population est la plus importante ?

# Question 3 
- Combien de pays ont Elisabeth II à la tête de leur gouvernement ?

- Quelle proporition des sujets de Sa Majesté ne parlent pas anglais ?
    - 78 % ou 83% ?

## Question 4 - passons à Pandas
Créer une DataFrame qui contient les informations suivantes par pays :
- le nom
- le code du pays
- le nombre de langues parlées
- le nombre de langues officielles
- la population
- le GNP
- l'espérance de vie

##### Indice : utiliser la commande pd.read_sql_query


Que dit la matrice de corrélation de ces variables ?